# Further Hypothesis Testing

In [ ]:
# Select this cell and type Ctrl-Enter to execute the code below.

library(tidyverse)

set_plot_dimensions <- function(width_choice, height_choice) {
    options(repr.plot.width=width_choice, repr.plot.height=height_choice)
}

cbPal <- c("#E69F00", "#56B4E9", "#009E73", "#F0E442", "#CC79A7", "#0072B2", "#D55E00")

set_plot_dimensions(5, 4)

In [ ]:
# You should see "Attaching packages" and some ticks by the packages loaded.
# The "Conflicts" aren't a problem.

# Other problems loading the library? Try running this cell.

install.packages('tidyverse')

library(tidyverse)


## 5 - Comparing distributions over a categorical variable

In [ ]:
# Run this cell to load the data.

data <- read_csv("stars.csv")

type_key <- c('Brown Dwarf', 'Red Dwarf', 'White Dwarf', 'Main Sequence', 'Supergiant','Hypergiant')
spectral_classes <- c('O','B','A','F','G','K','M')

data$type <- factor(data$type)
data$spectral_class <- factor(data$spectral_class, levels=spectral_classes)


Your undergraduate project student, Tunde, is looking at the distributions of spectral class (i.e. colour) for various types of star. 

He says that the bar charts for white dwarves (type 2) and main sequence stars (type 3) seem similar, but you are not so sure. 


In [ ]:

data %>% 
    filter(type %in% c(2,3)) %>% 
    select(type,spectral_class) %>%
    ggplot(aes(x=spectral_class, fill=type)) +
        geom_bar(position = position_dodge(preserve = "single")) +
        scale_fill_manual(values=cbPal[c(3,4)]) +
        scale_x_discrete(drop=FALSE)


### Question: do types 2 and 3 share the same distribution of spectral class?

To test differences in distributions over a *categorical variable* like spectral class, we can use [*Pearson's chi-squared test*](https://en.wikipedia.org/wiki/Chi-squared_test):

### Chi-squared test for statistical independence

#### Theory

$H_0$: Probability distribution for the categorical variable (e.g. spectral class) is independent of group (e.g. star type).

$H_1$: Probability distribution depends on group.

We need to find out whether the differences in observed frequencies between the two groups are small enough to have arisen by chance. We do this by constructing a [*contingency table*](https://en.wikipedia.org/wiki/Contingency_table) showing the observed frequency of each outcome for each of the two groups, and comparing to the *expected frequencies* under $H_0$.


The test statistic is

$$X^2 = \sum^k_{i=1}{\frac{(x_i-m_i)^2}{m_i}}$$,

where $k$ is the number of cells in the table and $x_i$ and $m_i$ are the observed and expected frequencies for each cell.

Under $H_0$, $X^2$ follows a [$\chi^2$ distribution](https://en.wikipedia.org/wiki/Chi-squared_distribution), which is parametrised by the number of degrees of freedom. A contingency table of size $a \times b$ has $(a-1)(b-1)$ degrees of freedom, i.e. the number of independent counts when row and column sums are held fixed.

#### Assumptions

- Sampling is random
- Each sample is independent of the others
- Expected frequency for each cell must be sufficiently large

The approximation to the $\chi^2$ distribution breaks down if expected frequencies are too low. It will normally be acceptable so long as no more than 20% of the events have expected frequencies below 5. Where there is only 1 degree of freedom (i.e. a 2 $\times$ 2 table), the approximation is not reliable if expected frequencies fall below 10. 


#### Application

We will set $\alpha=0.05$.

We begin by making a contingency table for the observed spectral classes.

In [ ]:
# OBSERVED

obs <-
    data %>% 
    filter(type %in% c(2,3)) %>% 
    select(type,spectral_class) %>%
    table(exclude=list("G","M",0,1,4,5))

print("OBSERVED:")

print(obs)

In [ ]:
print("row totals:")
tot_rows <- margin.table(obs,1)
show(tot_rows)

print("column totals:")
tot_cols <- margin.table(obs,2)
print(tot_cols)

print("total observations")
tot_obs <- sum(tot_rows)
print(tot_obs)

From the row and column totals, we can calculate the *expected* frequencies under $H_0$:

In [ ]:
print("EXPECTED:")
exp <- as.matrix(tot_rows) %*% t(as.matrix(tot_cols)) / tot_obs
exp

The expected values for classes K and O are too small. We can combine columns O, F and K to get all of the expected values over 5:

In [ ]:
# OBSERVED (combining columns)

AB <- obs[,c('A','B')]
OFK <- margin.table(obs[,c('O','F','K')],1)
new_obs <- cbind(AB,OFK)
print("OBSERVED:")
new_obs

In [ ]:
# EXPECTED (combining columns)

AB <- exp[,c('A','B')]
OFK <- margin.table(exp[,c('O','F','K')],1)
new_exp <- cbind(AB,OFK)
print("EXPECTED:")
new_exp

In R, the most convenient form of the chi-squared test for a contingency table is `chi2.test`, which just takes the table of observations as input and calculates the expected values and degrees of freedom accordingly.

In [ ]:
chisq.test(new_obs)

According to the chi-squared test, $p<\alpha$, so there is enough evidence to reject $H_0$:
it appears that white dwarves and main sequence stars follow different distributions for spectral class.

<br>

---